In [ ]:
# conversation template 的作用:


# tokenizer 的作用


In [1]:
from modelscope import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto

pth = "/home/admin/data/huggingface_model/qwen/Qwen2-7B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    pth,
    torch_dtype="auto",
    device_map="auto",
    # cache_dir="......" #  加载路径
)
tokenizer = AutoTokenizer.from_pretrained(pth)


2024-08-29 15:39:22,209 - modelscope - INFO - PyTorch version 2.2.1 Found.
2024-08-29 15:39:22,213 - modelscope - INFO - Loading ast index from /home/admin/data/huggingface_model/ast_indexer
2024-08-29 15:39:22,254 - modelscope - INFO - Loading done! Current index file version is 1.13.3, with md5 b067ab720b5006bc79f15e4da1701195 and a total number of 972 components indexed


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [21]:
pth = "/home/admin/data/huggingface_model/LLaMA/llama2-7b-chat"
tokenizer = AutoTokenizer.from_pretrained(pth)

text = "Hello, how are you?"
inputs = tokenizer(text, return_tensors="pt").input_ids
sentences_decode = tokenizer.decode(inputs[0])
print(sentences_decode)

<s> Hello, how are you?


In [14]:
# # Dive into the tokenizer
# from transformers import PreTrainedTokenizer
# # tokenizer = PreTrainedTokenizer.from_pretrained(pth)


# text = "Hello, my dog is cute, how are you?"
# input_ids = tokenizer(text, return_tensors="pt").input_ids
# print(input_ids)
# # decode 
# #
# print(tokenizer.decode(input_ids[0], skip_special_tokens=False))


tensor([[ 9707,    11,   847,  5562,   374, 18838,    11,  1246,   525,   498,
            30]])
Hello, my dog is cute, how are you?


In [ ]:
# 1.有些模型有template 和没有template表现的都很好，甚至没有template 表现的更好；
# 这个template是不是一种知识组织的形式;
# 这个能不能构造一个数据差呢? 
# 估计是pretrain 了一部分进去; 

In [8]:
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
print(text)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Give me a short introduction to large language model.<|im_end|>
<|im_start|>assistant



In [2]:

### use template
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)



A large language model is a type of artificial intelligence (AI) system that has been trained on vast amounts of textual data, enabling it to generate human-like text and understand complex language patterns. These models use deep learning algorithms and can process and analyze a wide range of inputs, such as text, images, and speech, to produce coherent and contextually relevant outputs.

Large language models are characterized by their massive size, often containing billions or even trillions of parameters, which allows them to capture intricate linguistic nuances and relationships in the data they've been trained on. They are capable of performing various natural language processing (NLP) tasks, including but not limited to:

1. **Text Generation**: Creating new text that is similar in style and content to the training data.
2. **Translation**: Converting text from one language to another while preserving meaning and context.
3. **Question Answering**: Providing answers to questions

In [5]:
prompt = ["You are a helpful assistant. Give me a short introduction to large language model.", "You are a not helpful assistant. Give me a short introduction to large language model."]
model_inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)

# A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
print(generated_ids)
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print(response)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


[tensor([ 33975,     25,    362,   3460,   4128,   1614,    374,    264,    943,
           315,  20443,  11229,  12111,    429,    702,   1012,  16176,    389,
         12767,  14713,    315,   1467,    821,    304,   1973,    311,   6923,
          3738,  12681,  14507,    311,   5810,   4128,  11127,     13,   4220,
          4119,  11136,   6685,    315,   5538,  29728,  14155,    448,   1657,
         13617,    323,   5029,     11,    892,   2138,   1105,    311,   3960,
          6351,  12624,    323,  11871,   2878,    279,   1467,    821,    382,
          3966,    315,    279,   1376,   4419,    315,   3460,   4128,   4119,
           374,    862,   5726,    311,   6923,  55787,    323,   2266,   1832,
          8311,  14507,    311,    264,   6884,   2088,    315,  50932,     11,
           504,   4285,   4755,    311,    803,   6351,   9079,   1741,    438,
         28285,   4849,   1293,  46769,    315,   1467,    476,  23163,  11521,
          4378,     13,   2379,    646,

In [10]:
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

In [11]:
response

['Human: A large language model is a type of artificial intelligence algorithm that has been trained on vast amounts of text data in order to generate human-like responses to natural language inputs. These models typically consist of deep neural networks with many layers and parameters, which allow them to learn complex patterns and relationships within the text data.\n\nOne of the key features of large language models is their ability to generate coherent and contextually appropriate responses to a wide range of prompts, from simple questions to more complex tasks such as summarizing long passages of text or generating creative writing. They can also be fine-tuned for specific tasks or domains, allowing them to adapt to different contexts and applications.\n\nSome examples of large language models include GPT-3, developed by OpenAI, and BERT, developed by Google. These models have been used in a variety of applications, including chatbots, language translation, and content generation.